# Transforming `OpenFermion` qubit operators to `QuTip` objects

## Prerequisite

Installation cells for Google Colab users.

In [1]:
!pip install qutip
!pip install openfermion

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 20.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 66.8 MB/s eta 0:00:00
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13821 sha256=f58a71b0bf71fdcda7f00d78b2761ea34f470add413c6700135f9a7a63e9b5be
  Stored in directory: /root/.cache/pip/wheels/90/7c/45/18a0671e3c3316966ef7ed9ad2b3f3300a7e41d3421a44e799
Successfully built pubchempy


Import libaries.

In [4]:
import numpy as np
import qutip as qt

from openfermion.ops import FermionOperator, QubitOperator

## Tools

In [7]:
def max_qubits(qubit_operator):
    max_qubit_index = 0
    for term in qubit_operator.terms:
        for qubit, _ in term:
            max_qubit_index = max(max_qubit_index, qubit)
    return max_qubit_index + 1

In [8]:
def extract_opf_qubitop(op):
    """
    Extract coefficients and Pauli words from a `QubitOperator`.

    Argument:
    op -- OpenFermion qubit operator
    """
    # Initialize
    coeffs = []
    pstrings = []

    # Loop
    for term, coeff in op.terms.items():
        pstring = ''.join([f"{p}{i}" for i, p in sorted(term)])
        coeffs.append(coeff)
        pstrings.append(pstring)

    return coeffs, pstrings

In [9]:
def updated_string(pword, nqubits):
    """
    Update OpenFermion Pauli words.

    Arguments:
    pword -- Pauli word as OpenFermion string
    nqubits -- number of qubits
    """
    # Check
    if len(pword) == 0:
        pstring = []
        for j in range(nqubits):
            pstring.append('I')
        return pstring

    # Non-trivial qubits
    lstr = len(pword) // 2
    ntind = []
    for j in range(lstr):
        ntind.append( int(pword[ (2 * j) + 1 ]) )

    # Updated Pauli string
    pstring = []
    for j in range(nqubits):
        A = 0
        for k in range(lstr):
            if j == ntind[k]:
                pstring.append(pword[2 * k])
                A += 1
        if A == 0:
            pstring.append('I')

    return pstring

In [10]:
def build_op_pauli(pstr):
    """
    Build a QuTip version of a Pauli operator.

    Argument:
    pstr -- Pauli operator as a string
    """
    if pstr == 'I':
        op = qt.qeye(2)
    elif pstr == 'X':
        op = qt.sigmax()
    elif pstr == 'Y':
        op = qt.sigmay()
    elif pstr == 'Z':
        op = qt.sigmaz()

    return op

In [11]:
def build_op_pword(pstring, nqubits):
    """
    Build a qubit operator for a Pauli word.

    Arguments:
    pstring -- Pauli word as string
    """
    op = build_op_pauli(pstring[0])
    for j in range(1, nqubits):
        op = qt.tensor(op, build_op_pauli(pstring[j]))

    return op

In [12]:
def build_ham_qutip(coeffs, pwords, nqubits):
    """
    Build a qubit Hamiltonian using QuTip.

    Arguments:
    coeffs -- Hamiltonian coefficients
    pwords -- set of Pauli words as OpenFermion strings
    nqubits -- number of qubits
    """
    # Check
    if len(coeffs) != len(pwords):
        raise ValueError("Lengths of coeffs and pwords do not match.")

    # Hamiltonian
    ham = ( coeffs[0] * build_op_pword( updated_string(pwords[0], nqubits), nqubits ))
    for j in range(1, len(pwords)):
        ham += ( coeffs[j] * build_op_pword( updated_string(pwords[j], nqubits), nqubits ))

    return ham

## Explore

Let us say we have a four-qubit operator $X_0 Y_1 X_2 Z_3$, where the subscripts indicate qubit indices.

In [22]:
op1 = QubitOperator('X0 Y1 X2 Z3')
C1, S1 = extract_opf_qubitop(op1)

In [23]:
updated_string(S1[0], 4)

['X', 'Y', 'X', 'Z']

We can have additional identities if we want.

In [25]:
updated_string(S1[0], 6)

['X', 'Y', 'X', 'Z', 'I', 'I']

Let us now build an `OpenFermion` Hamiltonian.

In [26]:
hamiltonian = 0.5 * QubitOperator('X0 X5') + 0.3 * QubitOperator('Z0')
hamiltonian

0.5 [X0 X5] +
0.3 [Z0]

Get the coefficients and each term.

In [29]:
C2, S2 = extract_opf_qubitop(hamiltonian)

We want a system with six qubits.

In [27]:
nqubits = 6

The first term.

In [31]:
updated_string(S2[0], nqubits)

['X', 'I', 'I', 'I', 'I', 'X']

Get the `QuTip` Hamiltonian.

In [33]:
HQ = build_ham_qutip(C2, S2, nqubits)
HQ

Quantum object: dims=[[2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2]], shape=(64, 64), type='oper', dtype=CSR, isherm=True
Qobj data =
[[ 0.3  0.   0.  ...  0.   0.   0. ]
 [ 0.   0.3  0.  ...  0.   0.   0. ]
 [ 0.   0.   0.3 ...  0.   0.   0. ]
 ...
 [ 0.   0.   0.  ... -0.3  0.   0. ]
 [ 0.   0.   0.  ...  0.  -0.3  0. ]
 [ 0.   0.   0.  ...  0.   0.  -0.3]]

In [35]:
ham_qutip = HQ.full()
ham_qutip[:, 0]

array([0.3+0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
       0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
       0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
       0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
       0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0.5+0.j, 0. +0.j,
       0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
       0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
       0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
       0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j, 0. +0.j,
       0. +0.j])